<a href="https://colab.research.google.com/github/pritamudugade/Data-Science/blob/main/Neural_network_custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras import layers, Model

In [4]:
import cv2
import glob
import os
import numpy as np

# Define the size of the synthetic images
image_height = 256
image_width = 256

# Define the number of classes
num_classes = 5

# Create an empty list to store the synthetic dataset
synthetic_dataset = []

# Path to your annotation folder
annotation_folder = "/content/drive/MyDrive/IntruderDetectionYOLOv8.v1i.yolov7pytorch/train/labels"

# Load real images from your dataset
image_paths = glob.glob("/content/drive/MyDrive/IntruderDetectionYOLOv8.v1i.yolov7pytorch/train/*.jpg")

# Generate the synthetic dataset
for image_path in image_paths:
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print("Error loading image:", image_path)
        continue
    image = cv2.resize(image, (image_width, image_height))  # Resize the image to the desired dimensions

    # Load annotation file for the current image
    annotation_file = os.path.join(annotation_folder, os.path.basename(image_path).replace(".jpg", ".txt"))

    # Read bounding box coordinates and class labels from the annotation file
    bounding_boxes = []
    class_labels = []
    with open(annotation_file, "r") as f:
        lines = f.readlines()
        for line in lines:
            parts = line.strip().split()
            class_label = int(parts[0])  # Assuming class label is the first value in the annotation line
            x_center = float(parts[1]) * image_width
            y_center = float(parts[2]) * image_height
            width = float(parts[3]) * image_width
            height = float(parts[4]) * image_height
            x1 = int(x_center - width / 2)
            y1 = int(y_center - height / 2)
            bounding_boxes.append([x1, y1, width, height])
            class_labels.append(class_label)

    # Append the image, bounding boxes, and class labels to the synthetic dataset
    synthetic_dataset.append({"image": image, "bounding_boxes": bounding_boxes, "class_labels": class_labels})

# Convert the synthetic dataset to NumPy arrays
X = np.array([data["image"] for data in synthetic_dataset])
y_bboxes = np.array([data["bounding_boxes"] for data in synthetic_dataset])
y_labels = np.array([data["class_labels"] for data in synthetic_dataset])

# Now you can use X, y_bboxes, and y_labels for training your object detection model
# Make sure to preprocess the data and annotations before feeding them to the model


In [5]:
# Define the CNN-based object detection model
class ObjectDetectionModel(Model):
    def __init__(self, num_classes, num_anchors):
        super(ObjectDetectionModel, self).__init__()
        # Define the layers for your model
        # Replace the following lines with your custom CNN architecture for object detection
        self.conv1 = layers.Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')
        self.pool1 = layers.MaxPooling2D(pool_size=(2, 2))
        self.conv2 = layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')
        self.pool2 = layers.MaxPooling2D(pool_size=(2, 2))
        self.flatten = layers.Flatten()
        self.fc1 = layers.Dense(128, activation='relu')
        self.output_layer = layers.Dense(num_classes + 4 * num_anchors, activation='linear')

    def call(self, x):
        # Define the forward pass of your model
        # Replace the following lines with your custom model architecture
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.output_layer(x)

        return x

In [20]:
# Create an instance of the model
num_classes = 4  # Replace with the number of classes in your dataset
num_anchors = 4   # Replace with the number of anchors used in the detection
model = ObjectDetectionModel(num_classes=num_classes, num_anchors=num_anchors)
model = ObjectDetectionModel(num_classes, num_anchors)


In [7]:
# Define the loss function and optimizer
loss_function = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam()

In [14]:
import tensorflow as tf

def custom_object_detection_loss(y_true, y_pred):
    # Split y_true into classification and regression targets
    true_classes = y_true[:, :num_classes]
    true_boxes = y_true[:, num_classes:]

    # Split y_pred into predicted classes and regression predictions
    pred_classes = y_pred[:, :num_classes]
    pred_boxes = y_pred[:, num_classes:]

    # Classification loss (example: categorical cross-entropy)
    class_loss = tf.keras.losses.CategoricalCrossentropy()(true_classes, pred_classes)

    # Regression loss (example: smooth L1 loss for bounding box regression)
    reg_loss = tf.keras.losses.Huber()(true_boxes, pred_boxes)

    # Combine the losses with appropriate weights
    total_loss = class_loss + reg_loss

    return total_loss


In [15]:
# Training loop
epochs = 50
batch_size = 32

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    for i in range(0, len(X), batch_size):
        batch_images = X[i:i+batch_size]
        batch_labels = y[i:i+batch_size]

        with tf.GradientTape() as tape:
            predictions = model(batch_images)
            loss = loss_function(batch_labels, predictions)

        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        print(f"Batch {i//batch_size+1}/{len(X)//batch_size}, Loss: {loss.numpy()}")


Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Epoch 14/50
Epoch 15/50
Epoch 16/50
Epoch 17/50
Epoch 18/50
Epoch 19/50
Epoch 20/50
Epoch 21/50
Epoch 22/50
Epoch 23/50
Epoch 24/50
Epoch 25/50
Epoch 26/50
Epoch 27/50
Epoch 28/50
Epoch 29/50
Epoch 30/50
Epoch 31/50
Epoch 32/50
Epoch 33/50
Epoch 34/50
Epoch 35/50
Epoch 36/50
Epoch 37/50
Epoch 38/50
Epoch 39/50
Epoch 40/50
Epoch 41/50
Epoch 42/50
Epoch 43/50
Epoch 44/50
Epoch 45/50
Epoch 46/50
Epoch 47/50
Epoch 48/50
Epoch 49/50
Epoch 50/50


In [21]:
# Save the trained model as a SavedModel
model.save('object_detection_model', save_format='tf')
print("Trained model saved.")

ValueError: ignored

In [17]:
model.save_weights('object_detection_model_weights.h5')
